In [41]:
from anthropic import Anthropic
from dotenv import load_dotenv
import os
import numpy as np

load_dotenv()

True

In [27]:
client = Anthropic(api_key = os.environ["ANTHROPIC_API_KEY"])

In [56]:
prompt_main = """You are a talented actor in a play called The Adding Machine. Your task is to take a description of the current scene, your character, and the previous lines read by the other actors and generate your character's next line. 
You will also be provided the next line spoken, make sure that your response makes sense not only in the context of the previous lines but also the line that follows yours. Your response will only be the line your character will say and nothing else.
Do not include sounds for your character to make in your response (for example, sighing, yawning, etc)."""

scene_desc = "Your character is sitting at a table next to Alice and Bob, fellow coworkers. You are deciding where to order lunch from."

char_desc = "You play Dave, a long-winded, rude tech executive who is a picky eater."

lines = """Alice: Ok, it's almost lunch time, are you guys in the mood for anything?
Bob: I'm down for anything, what do you think Dave?
Dave: 
Bob: I'm not too sure about that place, last time I got food poisoning
Dave: 
Alice: Ok, I'll  decide then. Let's go get Chipotle!
Dave:"""

In [80]:
split_lines = (lines.split("\n"))

In [81]:
your_line_idxs = [i for i in range(len(split_lines)) if len(split_lines[i].split(":")[1].strip()) == 0]

In [82]:
prompt = """
You are a talented actor in a play called The Adding Machine. Your task is to take a description of the current scene, your character, and the previous lines read by the other actors and generate your character's next line. 
You will also be provided the next line spoken, make sure that your response makes sense not only in the context of the previous lines but also the line that follows yours. Your response will only be the line your character will say and nothing else.
You must not include sounds for your character to make in your response (for example, *sigh*, sighing, yawning, etc).

Scene Description:
Your character is sitting at a table next to Alice and Bob, fellow coworkers. You are deciding where to order lunch from.

Character Description:
You play Dave, a long-winded, rude tech executive who is a picky eater.

Lines:
{lines}

Next Line:
{next_line}
"""

In [83]:
for i in range(len(your_line_idxs)):
    idx = your_line_idxs[i]
    prev_lines = np.array(split_lines)[list(range(0, idx+1))]
    try:
        next_line = split_lines[idx+1]
    except:
        next_line = "None"

    lines_template = "\n".join(prev_lines)
    p = prompt.format(lines=lines_template, next_line=next_line)

    message = client.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": p
            }
        ],
        temperature=0.9,
        top_p=0.5,
        model="claude-3-opus-20240229"
    )

    split_lines[your_line_idxs[i]] += message.content[0].text

In [84]:
split_lines

["Alice: Ok, it's almost lunch time, are you guys in the mood for anything?",
 "Bob: I'm down for anything, what do you think Dave?",
 'Dave: Well, I\'ve been thinking about this a lot, and I have to say, there\'s only one place that truly meets my discerning culinary standards. It\'s this little hole-in-the-wall joint called "The Greasy Spoon." Now, I know what you\'re thinking, but trust me, their food is absolutely exquisite. The chef is a true artist, and the way he crafts each dish is simply unparalleled. I won\'t settle for anything less than perfection, and "The Greasy Spoon" is the only place that consistently delivers. So, that\'s my vote, take it or leave it.',
 "Bob: I'm not too sure about that place, last time I got food poisoning",
 'Dave: Bob, your weak stomach is not my problem. I refuse to compromise my refined palate just because you can\'t handle a little culinary adventure. "The Greasy Spoon" is a hidden gem, and if you can\'t appreciate that, then maybe you should s

"Chipotle? Really? I suppose if you enjoy bland, mass-produced burritos filled with subpar ingredients, then by all means, let's go. But don't expect me to lower my standards to accommodate your pedestrian tastes."